🧠 Convolutional Neural Network (CNN)
CNNs are a specialized type of ANN designed to process grid-like data, especially images.
🧱 Core Layers:
- Convolutional Layer: Applies filters (kernels) to extract spatial features like edges, textures, and shapes.
- Pooling Layer: Reduces spatial dimensions (typically using max pooling) to lower computation and retain dominant features.
- Fully Connected Layer: Flattens features and classifies based on learned patterns.
- Activation Functions: Usually ReLU in hidden layers; Softmax in output for multi-class classification.
📌 Why Use CNNs?
- Preserve spatial hierarchies in images.
- Fewer parameters than fully connected ANNs, thanks to weight sharing in convolution.
- Highly effective in image classification, object detection, and computer vision tasks.



In [2]:
import zipfile
import os
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split

In [3]:
uploaded = files.upload()

with zipfile.ZipFile("Rice_Image_Dataset.zip", 'r') as zip_ref:
    zip_ref.extractall()

dataset_path = "/content/Rice_Image_Dataset"

Saving Rice_Image_Dataset.zip to Rice_Image_Dataset.zip


In [4]:
IMG_SIZE = (32, 32)
x = []
y = []
labels = sorted(os.listdir(dataset_path))
labels = [label for label in labels if label != 'Rice_Citation_Request.txt']
label_map = {label: idx for idx, label in enumerate(labels)}

for label in labels:
    folder = os.path.join(dataset_path, label)
    if not os.path.isdir(folder):
        continue
    for img_file in os.listdir(folder):
        img_path = os.path.join(folder, img_file)
        img = load_img(img_path, target_size=IMG_SIZE)
        img_array = img_to_array(img) / 255.0
        x.append(img_array)
        y.append(label_map[label])

x = np.array(x)
y = np.array(y)

In [5]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=42)

In [6]:
cnn = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(len(label_map), activation='softmax')
])
cnn.summary()

cnn.compile(optimizer='adam',
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy'])

cnn.fit(x_train, y_train, epochs=5)

y_pred_cnn = cnn.predict(x_test)
y_pred_cnn_classes = [np.argmax(i) for i in y_pred_cnn]


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 30, 30, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 13, 13, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 6, 6, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1152)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │        73,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 84,261 (329.14 KB)

 Trainable params: 84,261 (329.14 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 61s 31ms/step - accuracy: 0.9026 - loss: 0.2634
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 79s 30ms/step - accuracy: 0.9716 - loss: 0.0811
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 56s 30ms/step - accuracy: 0.9839 - loss: 0.0476
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 80s 29ms/step - accuracy: 0.9847 - loss: 0.0447
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 83s 30ms/step - accuracy: 0.9872 - loss: 0.0368
469/469 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step


In [7]:

print("\n CNN Classification Report:")
print(classification_report(y_test, y_pred_cnn_classes, target_names=label_map.keys()))



 CNN Classification Report:
              precision    recall  f1-score   support

     Arborio       0.99      0.97      0.98      3000
     Basmati       0.97      1.00      0.98      3000
      Ipsala       1.00      1.00      1.00      3000
     Jasmine       0.99      0.97      0.98      3000
   Karacadag       0.97      1.00      0.98      3000

    accuracy                           0.98     15000
   macro avg       0.98      0.98      0.98     15000
weighted avg       0.98      0.98      0.98     15000

